# Prompting Tutorial
## This is a tutorial for prompting. We will use a few shot sentiment analysis task to show how to do prompting.

In [121]:
# load packages
import torch
import pandas as pd
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForMaskedLM

In [117]:
# load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

model = AutoModelForMaskedLM.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### An Example of Zero-shot 

In [123]:
Text='The review is [MASK]. Review: It is a bad movie.'

In [124]:
encoding = tokenizer(Text, truncation=True, padding=True,return_tensors='pt')

In [126]:
outputs = model(encoding['input_ids'], attention_mask=encoding['attention_mask'])

In [127]:
logits=outputs.logits[0,4,:]

In [128]:
top_5_tokens = np.argsort(-logits.detach().numpy())[:5].tolist()

In [130]:
for token in top_5_tokens:
    print(f">>> {Text.replace(tokenizer.mask_token, tokenizer.decode([token]))}")

>>> The review is negative. Review: It is a bad movie.
>>> The review is positive. Review: It is a bad movie.
>>> The review is mixed. Review: It is a bad movie.
>>> The review is favorable. Review: It is a bad movie.
>>> The review is poor. Review: It is a bad movie.


### In this tutorial, we choose the IMDB Dataset as our dataset. It has 50000 movie reviews, 25000 for postive and 25000 for negative.

In [26]:
data=pd.read_csv('IMDB Dataset.csv')

In [27]:
data

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [28]:
# Add Prompt
review_lst=data['review'].to_list()
prompt=[]
for sentence in review_lst:
    new_string='The review is [MASK]. Review: '+sentence
    prompt.append(new_string)
data['prompt']=prompt

In [29]:
data

,review,sentiment,prompt
0,One of the other reviewers has mentioned that ...,positive,The review is [MASK]. Review: One of the other...
1,A wonderful little production. <br /><br />The...,positive,The review is [MASK]. Review: A wonderful litt...
2,I thought this was a wonderful way to spend ti...,positive,The review is [MASK]. Review: I thought this w...
3,Basically there's a family where a little boy ...,negative,The review is [MASK]. Review: Basically there'...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,The review is [MASK]. Review: Petter Mattei's ...
...,...,...,...
49995,I thought this movie did a down right good job...,positive,The review is [MASK]. Review: I thought this m...
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative,"The review is [MASK]. Review: Bad plot, bad di..."
49997,I am a Catholic taught in parochial elementary...,negative,The review is [MASK]. Review: I am a Catholic ...
49998,I'm going to have to disagree with the previou...,negative,The review is [MASK]. Review: I'm going to hav...


In [131]:
# Add targets. We want to predict the mask token to be postive or negative

In [30]:
tokenizer.decode([3893])

'positive'

In [31]:
tokenizer.decode([4997])

'negative'

In [32]:
data['target'] = data['sentiment'].replace(['positive','negative'], [3893,4997])

In [33]:
data

,review,sentiment,prompt,target
0,One of the other reviewers has mentioned that ...,positive,The review is [MASK]. Review: One of the other...,3893
1,A wonderful little production. <br /><br />The...,positive,The review is [MASK]. Review: A wonderful litt...,3893
2,I thought this was a wonderful way to spend ti...,positive,The review is [MASK]. Review: I thought this w...,3893
3,Basically there's a family where a little boy ...,negative,The review is [MASK]. Review: Basically there'...,4997
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,The review is [MASK]. Review: Petter Mattei's ...,3893
...,...,...,...,...
49995,I thought this movie did a down right good job...,positive,The review is [MASK]. Review: I thought this m...,3893
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative,"The review is [MASK]. Review: Bad plot, bad di...",4997
49997,I am a Catholic taught in parochial elementary...,negative,The review is [MASK]. Review: I am a Catholic ...,4997
49998,I'm going to have to disagree with the previou...,negative,The review is [MASK]. Review: I'm going to hav...,4997


### Create a subset with 2 samples one for positive and one for negative

In [107]:
one_shot=data.groupby('sentiment').tail(1)

In [108]:
one_shot

,review,sentiment,prompt,target
49995,I thought this movie did a down right good job...,positive,The review is [MASK]. Review: I thought this m...,3893
49999,No one expects the Star Trek movies to be high...,negative,The review is [MASK]. Review: No one expects t...,4997


### Create a subset with 32 samples 16 for positive and 16 for negative

In [35]:
few_shot=data.groupby('sentiment').head(16)

### Create  dataloaders

In [34]:
class CustomDataset():

    def __init__(self, dataframe, tokenizer):
        self.tokenizer = tokenizer
        self.prompts = list(dataframe['prompt'])
        self.targets = list(dataframe['target'])

    def __len__(self):
        return len(self.prompts)

    def __getitem__(self, index):

        
        inputs = self.tokenizer.encode_plus(
            self.prompts[index],
            None,
            add_special_tokens=True,
            max_length=512,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True
        )
        
        
        
        
        ids_content = inputs['input_ids']
        mask_content = inputs['attention_mask']
        token_type_ids_content = inputs["token_type_ids"]
        targets=self.targets[index]
        return torch.tensor(ids_content, dtype=torch.long),torch.tensor(mask_content, dtype=torch.long),torch.tensor(token_type_ids_content, dtype=torch.long),torch.tensor(targets, dtype=torch.long)

In [113]:
one_shot_set = CustomDataset(one_shot, tokenizer)

oneshotloader = DataLoader(one_shot_set, batch_size=2,shuffle=True)

In [116]:
test_set = CustomDataset(data, tokenizer)

testloader = DataLoader(test_set, batch_size=2,shuffle=False)

# Training and Evaluating functions

In [39]:
def training(train_loader, model, criterion2,optimizer):
    """one epoch training"""


    epoch_loss = 0

    epoch_acc = 0
   
    model.train()


    for idx, (input_ids,attention_mask,token_type_ids,targets) in enumerate(train_loader):


        
        if torch.cuda.is_available():
            input_ids = input_ids.cuda()
            
            attention_mask = attention_mask.cuda()
                
            
            targets= targets.cuda()


        # compute loss
        outputs = model(input_ids, attention_mask=attention_mask).logits
        pred=outputs[:,4,:]
        loss = criterion2(pred, targets)
        
        acc = calculate_accuracy(pred, targets)
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()


        # SGD
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    



    return epoch_loss / len(train_loader), epoch_acc / len(train_loader)

In [103]:
def evaluate(train_loader, model,  criterion2):
    epoch_loss = 0

    epoch_acc = 0

    model.eval()
    
    with torch.no_grad():
    
        for idx, (input_ids,attention_mask,token_type_ids,targets) in enumerate(train_loader):

            if torch.cuda.is_available():
                input_ids = input_ids.cuda()
            
                attention_mask = attention_mask.cuda()
                
                targets= targets.cuda()
            optimizer.zero_grad()
               
            outputs = model(input_ids, attention_mask=attention_mask).logits
        
            pred=outputs[:,4,:]
            

            loss = criterion2(pred, targets)
        

            
            acc = calculate_accuracy(pred, targets)
        
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(train_loader), epoch_acc / len(train_loader)

In [118]:
#calculate accuracy
def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
criterion2 = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=1e-6,
    )

In [ ]:
#training and evaluate
for i in range(300):


    print('epochs:'+ str(i+1))

    tr_loss,tr_acc=training(oneshotloader, model, criterion2,optimizer)
    print('training_loss:'+str(round(tr_loss, 5))+' acc:'+str(round(tr_acc, 5)))


    ts_loss,ts_acc=evaluate(testloader, model,  criterion2)
    print('ts_loss:'+str(round(ts_loss, 5))+' ts_acc:'+str(round(ts_acc, 5)))
    

### After 350 epochs, the accracy reachs 0.79 for testing with only 2 samples in training
### After 30 epochs, the accracy reachs 0.82 for testing with only 36 samples in training